In [1]:
import numpy as np
import pickle
import math
import pandas as pd

# Extracting Data 
    
    1) Merging batch_1, batch_2, batch_3, batch_4 and batch_5 data and their labels. Making (50000,3073) data matrix.
       Where first 3072 columns corresponds to image pixels and last column belongs to label. And train data is of size 50000.
    2)Then from batch_test its data and labels are merged and data matrix of size (10000,3073) is formed.
    3)Then cifar_2_data is subset of cifar_10 data which contains data which correspond to label 0 and 1 only.
    4)Then cifar_5_data is subset of cifar_10 data which contains data which correspond to label 0,1,2,3,4 only.
    5)Same is done for cifar_2_test and cifar_5_test, but with test data

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

batch_1 = unpickle('./image_data/cifar-10-batches-py/data_batch_1')
batch_2 = unpickle('./image_data/cifar-10-batches-py/data_batch_2')
batch_3 = unpickle('./image_data/cifar-10-batches-py/data_batch_3')
batch_4 = unpickle('./image_data/cifar-10-batches-py/data_batch_4')
batch_5 = unpickle('./image_data/cifar-10-batches-py/data_batch_5')
batch_test = unpickle('./image_data/cifar-10-batches-py/test_batch')

data_temp = np.concatenate([batch_1[b'data'],batch_2[b'data'],batch_3[b'data'],batch_1[b'data'],batch_5[b'data']],axis=0)
filename_temp = np.concatenate([batch_1[b'filenames'],batch_2[b'filenames'],batch_3[b'filenames'],batch_1[b'filenames'],batch_5[b'filenames']],axis=0)
label_temp = np.concatenate([batch_1[b'labels'],batch_2[b'labels'],batch_3[b'labels'],batch_1[b'labels'],batch_5[b'labels']],axis=0)
cifar_10_data = np.concatenate((data_temp,label_temp.reshape((-1,1))),axis=1)
cifar_10_data_test = np.concatenate([batch_test[b'data'],np.array(batch_test[b'labels']).reshape((-1,1))],axis=1)

cifar_2_data = cifar_10_data[cifar_10_data[:,-1]<= 1]                      
cifar_2_data_test = cifar_10_data_test[cifar_10_data_test[:,-1]<= 1]
cifar_5_data = cifar_10_data[cifar_10_data[:,-1]<= 4]
cifar_5_data_test = cifar_10_data_test[cifar_10_data_test[:,-1]<= 4]

# Calculating mean and std
    1) Mean of each channel is calculated from train data for cifar_2_data and cifar_5_data seperately.
    2) Similarly standard deviation of each channel is calculated from train data for cifar_2_data and cifar_5_data seperately.

In [3]:
cifar_2_data = cifar_2_data.astype(float)
cifar_2_data_test = cifar_2_data_test.astype(float)
cifar_5_data = cifar_5_data.astype(float)
cifar_5_data_test = cifar_5_data_test.astype(float)

r_mean = np.mean(cifar_2_data[:1024,:-1])
g_mean = np.mean(cifar_2_data[1024:2048,:-1])
b_mean = np.mean(cifar_2_data[2048:3072,:-1])
r_std = np.std(cifar_2_data[:1024,:-1])
g_std = np.std(cifar_2_data[1024:2048,:-1])
b_std = np.std(cifar_2_data[2048:3072,:-1])

r_mean_5 = np.mean(cifar_5_data[:1024,:-1])
g_mean_5 = np.mean(cifar_5_data[1024:2048,:-1])
b_mean_5 = np.mean(cifar_5_data[2048:3072,:-1])
r_std_5 = np.std(cifar_5_data[:1024,:-1])
g_std_5 = np.std(cifar_5_data[1024:2048,:-1])
b_std_5 = np.std(cifar_5_data[2048:3072,:-1])

# Normalizing Data
    1) Train Data and test data for both cifar_2 and cifar_5 is normalized by subtracting mean calculated above and diving it by standard deviation of each channel seperately.

In [4]:
cifar_2_data[:1024,:-1] = (cifar_2_data[:1024,:-1] - r_mean)/r_std
cifar_2_data[1024:2048,:-1] = (cifar_2_data[1024:2048,:-1] - g_mean)/g_std
cifar_2_data[2048:3072,:-1] = (cifar_2_data[2048:3072,:-1] - b_mean)/b_std

cifar_2_data_test[:1024,:-1] = (cifar_2_data_test[:1024,:-1] - r_mean)/r_std
cifar_2_data_test[1024:2048,:-1] = (cifar_2_data_test[1024:2048,:-1] - g_mean)/g_std
cifar_2_data_test[2048:3072,:-1] = (cifar_2_data_test[2048:3072,:-1] - b_mean)/b_std

cifar_5_data[:1024,:-1] = (cifar_5_data[:1024,:-1] - r_mean_5)/r_std_5
cifar_5_data[1024:2048,:-1] = (cifar_5_data[1024:2048,:-1] - g_mean_5)/g_std_5
cifar_5_data[2048:3072,:-1] = (cifar_5_data[2048:3072,:-1] - b_mean_5)/b_std_5

cifar_5_data_test[:1024,:-1] = (cifar_5_data_test[:1024,:-1] - r_mean_5)/r_std_5
cifar_5_data_test[1024:2048,:-1] = (cifar_5_data_test[1024:2048,:-1] - g_mean_5)/g_std_5
cifar_5_data_test[2048:3072,:-1] = (cifar_5_data_test[2048:3072,:-1] - b_mean_5)/b_std_5

# Input a set of training examples
    For each training example x: Set the corresponding input activation ax,1, and perform the following steps:
    Feedforward: For each l=2,3,…,L compute zx,l=wlax,l−1+bl and ax,l=σ(zx,l).
    Output error δx,L: Compute the vector δx,L=∇aCx⊙σ′(zx,L).
    Backpropagate the error: For each l=L−1,L−2,…,2 compute δx,l=((wl+1)Tδx,l+1)⊙σ′(zx,l).
    Gradient descent: For each l=L,L−1,…,2 update the weights according to the rule wl→wl−ηm∑xδx,l(ax,l−1)T, and the biases 
    according to the rule bl→bl−ηm∑xδx,l.
    reference :- http://neuralnetworksanddeeplearning.com/chap2.html
                 https://doug919.github.io/notes-on-backpropagation-with-cross-entropy/

In [9]:
class neural_network():
    
    def __init__(self,architecture,pretrained=False,model_path=None):
        
        if pretrained is False:
            self.bias_matrix = [np.random.randn(number_of_neuron)/math.sqrt(3072) for number_of_neuron in architecture[1:]]
            self.weights_matrix = [(np.random.randn(neuron_in_next_layers, neuron_in_previous_layers)/math.sqrt(3072))\
                                   for neuron_in_previous_layers, neuron_in_next_layers in zip(architecture[:-1], architecture[1:])]
        else:
            with open(model_path+'model_weight.pkl','rb') as f:
                self.weights_matrix = pickle.load(f)
            with open(model_path+'model_bais.pkl','rb') as b:
                self.bias_matrix = pickle.load(b)
            
        self.num_layers = len(architecture)
        self.architecture = architecture
    
    def fc_layer(self,input_):
        temp = input_.copy()
        count = 0 
        for bais,weights in zip(self.bias_matrix,self.weights_matrix):
            if count<len(bais)-1:
                temp = self.relu(np.matmul(weights,temp)+bais)
            else:
                temp = self.softmax(np.matmul(weights,temp)+bais)
            count += 1
        return temp 
    
    def relu(self,vector):
        temp = vector.copy()
        return np.where(temp>=0,temp,0.0)
    
    def softmax(self,vector):
        temp = vector.copy()
        temp = np.exp(temp)
        return temp/np.sum(temp)
    
    def cross_entropy(self,vector,label):
        temp1 = vector.copy()
        temp2 = np.zeros(temp1.shape[0])
        temp2[int(label)] = 1
        temp1 = np.clip(temp1,1e-2,1-(1e-2))
        return -1*np.sum(np.log(temp1)*temp2)/temp1.shape[0]
    
    def backpropagation(self, image, gt_truth):
        image_delta_w = [np.zeros((neuron_in_next_layers, neuron_in_previous_layers))\
                               for neuron_in_previous_layers, neuron_in_next_layers in zip(self.architecture[:-1], self.architecture[1:])]
        image_delta_b = [np.zeros(number_of_neuron) for number_of_neuron in self.architecture[1:]]
        
        temp = image.copy()
        z_layers=[]
        activation_layers=[]
        activation_layers.append(temp)
        count = 0 
        for bais,weights in zip(self.bias_matrix,self.weights_matrix):
            temp = np.matmul(weights,temp)+bais
            z_layers.append(temp)
            if count<len(bais)-1:
                temp = self.relu(temp)
            else:
                temp = self.softmax(temp)
                
            activation_layers.append(temp)
            count += 1
        
        one_hot_vector_label = np.zeros(temp.shape[0])
        one_hot_vector_label[int(gt_truth)] = 1
        del_cost_activation = temp - one_hot_vector_label
        image_delta_w[-1] = np.matmul(del_cost_activation.reshape((-1,1)), activation_layers[-2].reshape((1,-1)))
        image_delta_b[-1] = del_cost_activation.copy()
    
        
        for i in range(2,len(image_delta_w)+1):
            del_cost_activation_temp = del_cost_activation.copy()
            del_cost_activation = np.squeeze(np.matmul(del_cost_activation_temp.reshape((1,-1)),self.weights_matrix[-1*i+1]))
            del_cost_activation = np.where(z_layers[-1*i]>=0, del_cost_activation, 0)
            image_delta_w[-1*i] = np.matmul(del_cost_activation.reshape((-1,1)), activation_layers[-1*i-1].reshape((1,-1)))
            image_delta_b[-1*i] = del_cost_activation.copy()
        
        return image_delta_w, image_delta_b
    
    def train(self,train_data=[],number_of_epochs=5,learning_rate=0.01,batch_size=50,test_data=[],save_path = None):
        data = train_data.copy()
        previous_loss = float('Inf')
        train_continue = True
        early_stopping = np.zeros(3)
        count = 0
#         for epoch in range(number_of_epochs):
        while(train_continue==True):
            correct_prediction = 0
            training_loss = 0
            np.random.shuffle(data)
            
            for image in data:
                output_softmax = self.fc_layer(image[:3072])
                training_loss += self.cross_entropy(output_softmax,image[-1])
                output = np.argmax(output_softmax)
                if output == image[-1]:
                    correct_prediction += 1
                    
                Training_Accuracy = (correct_prediction/data.shape[0])*100
            
            if test_data!=[]:
                correct_prediction = 0
                testing_loss = 0
                for image in test_data:
                    output_softmax = self.fc_layer(image[:3072])
                    testing_loss += self.cross_entropy(output_softmax,image[-1])
                    output = np.argmax(output_softmax)
                    if output == image[-1]:
                        correct_prediction += 1
                Testing_Accuracy = (correct_prediction/test_data.shape[0])*100
                
                if testing_loss<previous_loss:
                    previous_loss = testing_loss
                    with open(save_path+'model_weight.pkl','wb') as f:
                        pickle.dump(self.weights_matrix, f)
                    with open(save_path+'model_bais.pkl','wb') as b:
                        pickle.dump(self.bias_matrix, b)
                
                if (testing_loss-previous_loss)>=0.1:
                    early_stopping[count%3] = 1
                else:
                    early_stopping[count%3] = 0
                
                if np.sum(early_stopping) == 3:
                    train_continue = False
                    
                count+=1
                
#                 print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3))
                print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3), ', Train Accuracy:', round(Training_Accuracy,3),'%',', Test Accuracy:', round(Testing_Accuracy,3),'%')
            
            else:
                print('epoch:',epoch ,', Train Loss:', round(training_loss,3))
                #print('epoch:',epoch ,', Train Loss:', round(training_loss,3),', Train Accuracy:', round(Training_Accuracy,3),'%')
    
            for batch_number in range(data.shape[0]//batch_size):
                
                batch_delta_w = [np.zeros((neuron_in_next_layers, neuron_in_previous_layers))\
                                 for neuron_in_previous_layers, neuron_in_next_layers in zip(self.architecture[:-1], self.architecture[1:])]
                batch_delta_b = [np.zeros(number_of_neuron) for number_of_neuron in self.architecture[1:]]
                
                for image in data[batch_size*batch_number:batch_size*(batch_number+1)]:
                    image_delta_w, image_delta_b = self.backpropagation(image[:3072], image[-1])
                
                    batch_delta_w = [batch_weight_delta+image_weight_delta for batch_weight_delta, image_weight_delta in zip(batch_delta_w, image_delta_w)]
                    batch_delta_b = [batch_bais_delta+image_bais_delta for batch_bais_delta, image_bais_delta in zip(batch_delta_b, image_delta_b)]
                    
                
                self.weights_matrix = [weight_layer-(learning_rate/batch_size)*weight_layer_delta
                            for weight_layer, weight_layer_delta in zip(self.weights_matrix, batch_delta_w)]
                self.bias_matrix = [bais_layer-(learning_rate/batch_size)*bais_layer_delta
                           for bais_layer, bais_layer_delta in zip(self.bias_matrix, batch_delta_b)]
            
                del batch_delta_w, batch_delta_b, image_delta_w, image_delta_b
    
    def evaluate(self,test_data):
        data = test_data.copy()
        testing_loss = 0
        correct_prediction = 0
        for image in data:
            output_softmax = self.fc_layer(image[:3072])
            testing_loss += self.cross_entropy(output_softmax,image[-1])
            output = np.argmax(output_softmax)
            if output == image[-1]:
                correct_prediction += 1
        print('Test Loss:', round(testing_loss,3),', Test Accuracy:', round((correct_prediction/data.shape[0])*100,3),'%')
            
       

In [10]:
np.random.seed(2)
my_network = neural_network([3072,100,2])
my_network.train(train_data=cifar_2_data, number_of_epochs=100, learning_rate=0.0001, batch_size=10,test_data=cifar_2_data_test,save_path='./model/2_')
# train_data=cifar_2_data, number_of_epochs=100, learning_rate=0.0001, batch_size=10,test_data=cifar_2_data_test

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


epoch: 1 , Train Loss: 7984.343 , Test Loss: 689.154 , Train Accuracy: 53.83 % , Test Accuracy: 55.55 %


KeyboardInterrupt: 

In [11]:
test_network = neural_network([3072,10,2],pretrained=True,model_path='./model/2_')
test_network.evaluate(test_data=cifar_2_data_test)

Test Loss: 689.154 , Test Accuracy: 55.55 %


In [13]:
np.random.seed(2)
my_network_5 = neural_network([3072,50,5])
my_network_5.train(train_data=cifar_5_data, number_of_epochs=100, learning_rate=0.00001, batch_size=10,test_data=cifar_5_data_test,save_path='./model/5_')
# train_data=cifar_5_data, number_of_epochs=100, learning_rate=0.00001, batch_size=10,test_data=cifar_5_data_test

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


epoch: 1 , Train Loss: 12748.378 , Test Loss: 3435.886 , Train Accuracy: 19.805 % , Test Accuracy: 17.1 %


KeyboardInterrupt: 

In [15]:
test_network_5 = neural_network([3072,10,5],pretrained=True,model_path='./model/5_')
test_network_5.evaluate(test_data=cifar_5_data_test)

Test Loss: 3435.886 , Test Accuracy: 17.1 %
